# Reacting to events

If a friend has high blood pressure we better let them know!

We'll need some stuff prom those previous notebooks...

In [109]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A list of sample messages
%store -r payloads
#A sample message payload
%store -r payload

Let's start with a very simple function that logs a simple message.


This is your code and could do anything: send an em-mail, a tweet, a phone call...

In [110]:
# %load hypertension.py
def on_hbp(event, context):
    name = event['name']
    print("High blood pressure detected. Take care {name}!")


To create a lambda function we need to:

- Create a zip package with the code

In [111]:
zip_file = f"{home}/aws_iot/hypertension{HM}.zip"
!zip -g {zip_file} hypertension.py
zip_file

	zip warning: /Users/faermanj/aws_iot/hypertension0545.zip not found or empty
  adding: hypertension.py (deflated 11%)


'/Users/faermanj/aws_iot/hypertension0545.zip'

- Create a role authorizing the function execution

In [34]:
print(open('lambda-trust.json').read())

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}


In [112]:
role_name = f"fn_hbp_role{HM}"
fn_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://lambda-trust.json \
    --query Role.Arn \
    --output text
fn_role_arn = fn_role_arn.s
fn_role_arn

'arn:aws:iam::030555009967:role/fn_hbp_role0545'

In [116]:
policy_arn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
!aws iam attach-role-policy \
    --role-name '{role_name}' \
    --policy-arn '{policy_arn}'

In [117]:
!echo {fn_role_arn}

arn:aws:iam::030555009967:role/fn_hbp_role0545


Create the function:

In [135]:
function_name=f"on_hbp{HM}"
function_name

'on_hbp0545'

In [118]:
function_arn = !aws lambda create-function \
    --function-name '{function_name}' \
    --zip-file "fileb://{zip_file}" \
    --role '{fn_role_arn}' \
    --handler hypertension.on_hbp \
    --runtime python3.6 \
    --timeout 30 \
    --memory-size 128 \
    --query FunctionArn \
    --output text
function_arn = function_arn.s
function_arn

'arn:aws:lambda:us-east-1:030555009967:function:on_hbp0545'

Now let's create the rule that tells AWS IoT to call that function when a message with high blood pressure arrives.

For that we need:
- The SQL statement capturing messages with the desired condition

In [119]:
sql = f"""
SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{topic_name}'
WHERE sistolic > 120 
    OR diastolic > 80
"""
print(sql)


SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM 'notebook-topic'
WHERE sistolic > 120 
    OR diastolic > 80



- Authorize the rule to invoke the lambda function

https://docs.aws.amazon.com/iot/latest/developerguide/lambda-rule.html

In [126]:
import json
rule = {
        "sql": sql, 
        "ruleDisabled": False,
        "awsIotSqlVersion": "2016-03-23",
        "actions": [{
            "lambda": {
                "functionArn": function_arn
             }
        }]}
rule = json.dumps(rule)
rule_file=f"{home}/rule.json" 
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

'/Users/faermanj/rule.json'

In [127]:
print(open(rule_file).read())

{"sql": "\nSELECT name, \n    time_stamp, \n    sistolic, \n    diastolic, \n    id, \n    timestamp() as recv_stamp \nFROM 'notebook-topic'\nWHERE sistolic > 120 \n    OR diastolic > 80\n", "ruleDisabled": false, "awsIotSqlVersion": "2016-03-23", "actions": [{"lambda": {"functionArn": "arn:aws:lambda:us-east-1:030555009967:function:on_hbp0545"}}]}


In [128]:
rule_name=f"bp2lambda_rule{HM}"
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}
rule_name


An error occurred (ResourceAlreadyExistsException) when calling the CreateTopicRule operation: Topic Rule 'bp2lambda_rule0545' already exists for account


'bp2lambda_rule0545'

In [158]:
rule_arn=!aws iot get-topic-rule --rule-name '{rule_name}' --query ruleArn --output text
rule_arn = rule_arn.s
rule_arn

'arn:aws:iot:us-east-1:030555009967:rule/bp2lambda_rule0545'

In [149]:
account_id=!aws sts get-caller-identity --query Account --output text
account_id =account_id.s
account_id

'030555009967'

In [159]:
unique_id = f"unique{HM}" 
unique_id

'unique0545'

In [161]:
!aws lambda add-permission \
    --function-name "{function_name}" \
    --principal iot.amazonaws.com \
    --source-arn '{rule_arn}' \
    --source-account "{account_id}" \
    --statement-id "{unique_id}" \
    --action "lambda:InvokeFunction"

{
    "Statement": "{\"Sid\":\"unique0545\",\"Effect\":\"Allow\",\"Principal\":{\"Service\":\"iot.amazonaws.com\"},\"Action\":\"lambda:InvokeFunction\",\"Resource\":\"arn:aws:lambda:us-east-1:030555009967:function:on_hbp0545\",\"Condition\":{\"StringEquals\":{\"AWS:SourceAccount\":\"030555009967\"},\"ArnLike\":{\"AWS:SourceArn\":\"arn:aws:iot:us-east-1:030555009967:rule/bp2lambda_rule0545\"}}}"
}


In [162]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id=f"NotebookCient{HM}"
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [167]:
import random
payload = random.choice(payloads)
print(payload)
mqtt.publish(topic_name, payload , QoS = 0)

{"id": "baacda63-26a3-4a24-941c-dfe8ed2f4485", "name": "Clara Oswald", "sistolic": 138, "diastolic": 98, "time_stamp": 1517405078085}


True

In [168]:
log_group=f"/aws/lambda/{function_name}"
!aws logs describe-log-groups --log-group-name-prefix {log_group}

{
    "logGroups": [
        {
            "logGroupName": "/aws/lambda/on_hbp0545",
            "creationTime": 1517407363376,
            "metricFilterCount": 0,
            "arn": "arn:aws:logs:us-east-1:030555009967:log-group:/aws/lambda/on_hbp0545:*",
            "storedBytes": 0
        }
    ]
}


In [169]:
log_stream = !aws logs describe-log-streams \
    --log-group-name '{log_group}' \
    --order-by LastEventTime \
    --max-items 1  \
    --query 'logStreams[0].logStreamName'\
    --output text
log_stream = log_stream.s
log_stream

'2018/01/31/[$LATEST]f538993e78054058a578d609f5426bb2'

In [170]:
!aws logs get-log-events \
    --log-group-name '{log_group}' \
    --log-stream-name '{log_stream}' \
    --query 'events[].message' \
    --output text 

START RequestId: 68186871-068f-11e8-a25c-c1f18b0cc7b9 Version: $LATEST
	High blood pressure detected. Take care {name}!
	END RequestId: 68186871-068f-11e8-a25c-c1f18b0cc7b9
	REPORT RequestId: 68186871-068f-11e8-a25c-c1f18b0cc7b9	Duration: 0.31 ms	Billed Duration: 100 ms 	Memory Size: 128 MB	Max Memory Used: 22 MB	
	START RequestId: 69401fa2-068f-11e8-a2a4-d5bfb547d840 Version: $LATEST
	High blood pressure detected. Take care {name}!
	END RequestId: 69401fa2-068f-11e8-a2a4-d5bfb547d840
	REPORT RequestId: 69401fa2-068f-11e8-a2a4-d5bfb547d840	Duration: 0.30 ms	Billed Duration: 100 ms 	Memory Size: 128 MB	Max Memory Used: 22 MB	
	START RequestId: 6a43ac31-068f-11e8-ae0a-ff4e85248d6f Version: $LATEST
	High blood pressure detected. Take care {name}!
	END RequestId: 6a43ac31-068f-11e8-ae0a-ff4e85248d6f
	REPORT RequestId: 6a43ac31-068f-11e8-ae0a-ff4e85248d6f	Duration: 0.30 ms	Billed Duration: 100 ms 	Memory Size: 128 MB	Max Memory Used: 22 MB	

[Errno 54] Connection reset by peer
[Errno 54] Co

In [ ]:
mqtt.disconnect()